<a href="https://colab.research.google.com/github/perezsam/DM19-Lab2-Homework/blob/master/simple_text_preprocessing_TweetTokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load raw data from Kaggle ##

In [0]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

In [0]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 64 Dec 19 14:26 kaggle.json


In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!mkdir data

In [0]:
cd data

/content/data


In [0]:
# Copy the private data set locally.
!kaggle datasets download -d cssamuel/dmlab2

 99% 148M/149M [00:04<00:00, 20.4MB/s]
100% 149M/149M [00:04<00:00, 32.1MB/s]


In [0]:
!unzip dmlab2.zip

Archive:  dmlab2.zip
  inflating: data_identification.csv  
  inflating: emotion.csv             
  inflating: sampleSubmission.csv    
  inflating: tweets_DM.json          


In [0]:
cd ..

/content


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

import json
from pandas.io.json import json_normalize

Using TensorFlow backend.


In [0]:
# read data files
raw_data=pd.read_json("./data/tweets_DM.json",lines=True)
tweets=json_normalize(data=raw_data['_source'])
identify=pd.read_csv("./data/data_identification.csv")
emotion=pd.read_csv("./data/emotion.csv")

In [0]:
# rename column names
tweets=tweets.rename(index=str,columns={"tweet.text":"text", "tweet.tweet_id":"tweet_id",
                                       "tweet.hashtags":"hashtags"})
# add identify tags to dataframe
tweets=pd.merge(tweets,identify, on="tweet_id")

#get training set and test set
train_df=tweets[tweets["identification"] == "train"]
test_df=tweets[tweets["identification"] == "test"]

#add emotion column
train_df=pd.merge(train_df,emotion, on="tweet_id")
#test_df["emotion"]=""

#drop identification tags
train_df.drop(columns=["identification"],inplace=True)
test_df.drop(columns=["identification"],inplace=True)

#use tweet_id as index
#train_df.set_index("tweet_id",inplace=True)
#test_df.set_index("tweet_id",inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
train_df

,hashtags,tweet_id,text,emotion
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",anticipation
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",sadness
2,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,fear
3,"[authentic, LaughOutLoud]",0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,joy
4,[],0x2c91a8,Still waiting on those supplies Liscus. <LH>,anticipation
...,...,...,...,...
1455558,"[NoWonder, Happy]",0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,joy
1455559,[],0x38959e,In every circumtance I'd like to be thankful t...,joy
1455560,[blessyou],0x2cbca6,there's currently two girls walking around the...,joy
1455561,[],0x24faed,"Ah, corporate life, where you can date <LH> us...",joy


In [0]:
test_df

,hashtags,tweet_id,text
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
4,[],0x2de201,"""Trust is not the same as faith. A friend is s..."
9,"[materialism, money, possessions]",0x218443,When do you have enough ? When are you satisfi...
30,"[GodsPlan, GodsWork]",0x2939d5,"God woke you up, now chase the day #GodsPlan #..."
33,[],0x26289a,"In these tough times, who do YOU turn to as yo..."
...,...,...,...
1867525,[],0x2913b4,"""For this is the message that ye heard from th..."
1867529,[],0x2a980e,"""There is a lad here, which hath five barley l..."
1867530,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...
1867531,[],0x29d0cb,I swear all this hard work gone pay off one da...


In [0]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
s1 = '@remy: This is waaaaayyyy too much for you!!!!!!'
tknzr.tokenize(s1)

[':', 'This', 'is', 'waaayyy', 'too', 'much', 'for', 'you', '!', '!', '!']

Quick text cleaning for training dataset using TweetTokenizer

In [0]:
train_clean_text = []
for (i, row) in enumerate(train_df.values):

  train_tweet_id = row[1]
  train_tweet = row[2]
  emotion = row[3]

  train_tokenized_tweet = tknzr.tokenize(train_tweet)
  train_tokenized_tweet_str = ' '.join([str(elem) for elem in train_tokenized_tweet])

  train_clean_text_tweet = (train_tweet_id, train_tokenized_tweet_str, emotion)

  train_clean_text.append(train_clean_text_tweet)

In [0]:
print(len(train_clean_text))

1455563


In [0]:
# create training DataFrame from extracted list
clean_text_train_df = pd.DataFrame(train_clean_text, columns = ["tweet_id", "clean_text", "emotion"])  

# set id as the index
clean_text_train_df.set_index("tweet_id",inplace=True)

clean_text_train_df

,clean_text,emotion
tweet_id,,
0x376b20,"People who post "" add me on #Snapchat "" must b...",anticipation
0x2d5350,"As we see , Trump is dangerous to #freepress a...",sadness
0x1cd5b0,Now ISSA is stalking Tasha 😂 😂 😂 <LH>,fear
0x1d755c,Thx for the BEST TIME tonight . What stories !...,joy
0x2c91a8,Still waiting on those supplies Liscus . <LH>,anticipation
...,...,...
0x321566,I'm SO HAPPY ! ! ! #NoWonder the name of this ...,joy
0x38959e,In every circumtance I'd like to be thankful t...,joy
0x2cbca6,there's currently two girls walking around the...,joy


In [0]:
clean_text_train_df.to_csv("clean_text_training.csv")

In [0]:
pd.read_csv("clean_text_training.csv")

,tweet_id,clean_text,emotion
0,0x376b20,"People who post "" add me on #Snapchat "" must b...",anticipation
1,0x2d5350,"As we see , Trump is dangerous to #freepress a...",sadness
2,0x1cd5b0,Now ISSA is stalking Tasha 😂 😂 😂 <LH>,fear
3,0x1d755c,Thx for the BEST TIME tonight . What stories !...,joy
4,0x2c91a8,Still waiting on those supplies Liscus . <LH>,anticipation
...,...,...,...
1455559,0x321566,I'm SO HAPPY ! ! ! #NoWonder the name of this ...,joy
1455560,0x38959e,In every circumtance I'd like to be thankful t...,joy
1455561,0x2cbca6,there's currently two girls walking around the...,joy
1455562,0x24faed,"Ah , corporate life , where you can date <LH> ...",joy


Quick text cleaning for test dataset using TweetTokenizer

In [0]:
test_clean_text = []
for (i, row) in enumerate(test_df.values):

  test_tweet_id = row[1]
  test_tweet = row[2]
  #emotion = row[3]

  test_tokenized_tweet = tknzr.tokenize(test_tweet)
  test_tokenized_tweet_str = ' '.join([str(elem) for elem in test_tokenized_tweet])

  test_clean_text_tweet = (test_tweet_id, test_tokenized_tweet_str)

  test_clean_text.append(test_clean_text_tweet)

In [0]:
print(len(test_clean_text))

411972


In [0]:
# create testing DataFrame from extracted list
clean_text_test_df = pd.DataFrame(test_clean_text, columns = ["tweet_id", "clean_text"])  

# set id as the index
clean_text_test_df.set_index("tweet_id",inplace=True)

clean_text_test_df

,clean_text
tweet_id,
0x28b412,"Confident of your obedience , I write to you ,..."
0x2de201,""" Trust is not the same as faith . A friend is..."
0x218443,When do you have enough ? When are you satisfi...
0x2939d5,"God woke you up , now chase the day #GodsPlan ..."
0x26289a,"In these tough times , who do YOU turn to as y..."
...,...
0x2913b4,""" For this is the message that ye heard from t..."
0x2a980e,""" There is a lad here , which hath five barley..."
0x316b80,When you buy the last 2 tickets remaining for ...


In [0]:
clean_text_test_df.to_csv("clean_text_testing.csv")

In [0]:
pd.read_csv("clean_text_testing.csv")

,tweet_id,clean_text
0,0x28b412,"Confident of your obedience , I write to you ,..."
1,0x2de201,""" Trust is not the same as faith . A friend is..."
2,0x218443,When do you have enough ? When are you satisfi...
3,0x2939d5,"God woke you up , now chase the day #GodsPlan ..."
4,0x26289a,"In these tough times , who do YOU turn to as y..."
...,...,...
411967,0x2913b4,""" For this is the message that ye heard from t..."
411968,0x2a980e,""" There is a lad here , which hath five barley..."
411969,0x316b80,When you buy the last 2 tickets remaining for ...
411970,0x29d0cb,I swear all this hard work gone pay off one da...


Extracting train.csv file for BERT

In [0]:
df_model = pd.read_csv("clean_text_training.csv")

In [0]:
clean_text_bert = df_model[['tweet_id', 'clean_text']]

In [0]:
# get dummies series from emotion column
s = pd.Series(list(df_model['emotion']))
one_hot_emotion = pd.get_dummies(s)

In [0]:
one_hot_emotion

,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0
2,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
1455559,0,0,0,0,1,0,0,0
1455560,0,0,0,0,1,0,0,0
1455561,0,0,0,0,1,0,0,0
1455562,0,0,0,0,1,0,0,0


In [0]:
# add one-hot encoded emotions to the training DataFrame
bert_train_df = pd.concat([clean_text_bert, one_hot_emotion.reindex(clean_text_bert.index)], axis=1)

In [0]:
bert_train_df

,tweet_id,clean_text,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,0x376b20,"People who post "" add me on #Snapchat "" must b...",0,1,0,0,0,0,0,0
1,0x2d5350,"As we see , Trump is dangerous to #freepress a...",0,0,0,0,0,1,0,0
2,0x1cd5b0,Now ISSA is stalking Tasha 😂 😂 😂 <LH>,0,0,0,1,0,0,0,0
3,0x1d755c,Thx for the BEST TIME tonight . What stories !...,0,0,0,0,1,0,0,0
4,0x2c91a8,Still waiting on those supplies Liscus . <LH>,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1455559,0x321566,I'm SO HAPPY ! ! ! #NoWonder the name of this ...,0,0,0,0,1,0,0,0
1455560,0x38959e,In every circumtance I'd like to be thankful t...,0,0,0,0,1,0,0,0
1455561,0x2cbca6,there's currently two girls walking around the...,0,0,0,0,1,0,0,0
1455562,0x24faed,"Ah , corporate life , where you can date <LH> ...",0,0,0,0,1,0,0,0


In [0]:
bert_train_df.to_csv("clean_text_tweets.csv", index=False)

In [0]:
pd.read_csv("clean_text_tweets.csv")

,tweet_id,clean_text,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,0x376b20,"People who post "" add me on #Snapchat "" must b...",0,1,0,0,0,0,0,0
1,0x2d5350,"As we see , Trump is dangerous to #freepress a...",0,0,0,0,0,1,0,0
2,0x1cd5b0,Now ISSA is stalking Tasha 😂 😂 😂 <LH>,0,0,0,1,0,0,0,0
3,0x1d755c,Thx for the BEST TIME tonight . What stories !...,0,0,0,0,1,0,0,0
4,0x2c91a8,Still waiting on those supplies Liscus . <LH>,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1455559,0x321566,I'm SO HAPPY ! ! ! #NoWonder the name of this ...,0,0,0,0,1,0,0,0
1455560,0x38959e,In every circumtance I'd like to be thankful t...,0,0,0,0,1,0,0,0
1455561,0x2cbca6,there's currently two girls walking around the...,0,0,0,0,1,0,0,0
1455562,0x24faed,"Ah , corporate life , where you can date <LH> ...",0,0,0,0,1,0,0,0
